# Задание 1

### Составьте список KPI метрик необходимых для понимания качества интернет-казино.Ответ аргументируйте.

**1. Чистый игровой доход (NGR)**

Чистый игровой доход отражает реальную прибыль казино, очищенную от всех роялти и иных операционных расходов. Для любого бизнеса будет в приоритете срабатывать в плюс.

**2. Player Retention**

Важно, чтобы игрок задерживался в игре, как можно дольше и приносил прибыль. Таким образом маркетинговые затраты будут распределяться на большее количество игроков.

**3. Общая сумма депозитов (TDS)**

Если данная сумма активно повышается, то это говорит про отличную стратегию проекта.

**4. Conversion Rate (CR):**

Помогает оценить, насколько успешно казино привлекает платящих игроков.

**5. Lifetime Value (LTV):**

Оценка средней прибыли, которую приносит один игрок за время своей активности. Показывает долгосрочную доходность.

# Задание 2

● который посчитает количество уникальных устройств и уникальных пользователей по месяцам в период с Июля-2023 по Декабрь-2023
(включительно);

 SELECT toMonth(event_timestamp) AS month
        ,COUNT(distinct gadget_id) AS unique_gadgets
        ,COUNT(distinct player_id) AS unique_users
 FROM event_log 
 WHERE event_timestamp BETWEEN '2023-07-01' and '2024-01-01'
 GROUP BY month
 ORDER BY month;

 ● который выведет ТОП-5 стран по количеству зарегистрированных пользователей в 2023 году;

 SELECT country_code
        ,COUNT(DISTINCT player_id) AS player_count
 FROM user_info
 WHERE record_created BETWEEN '2023-01-01' AND '2024-01-01'
 GROUP BY country_code
 ORDER BY player_count DESC
 LIMIT 5;

● который выведет ТОП-5 пользователей по общей сумме платежей,и по каждой стране за 2023 год;

 WITH payment AS (
    SELECT 
        player_id
        ,country_code
        ,SUM(amount_RUB) AS payment_sum
    FROM payment_records
    JOIN user_info USING (player_id)
    WHERE payment_completed BETWEEN '2023-01-01' AND '2024-01-01'
          AND payment_successful = 1
    GROUP BY player_id, country_code
)
    
 SELECT 
    country_code
    ,player_id
    ,total_payment
 FROM payment 
 ORDER BY total_payment DESC, country_code
 LIMIT 5;

● который посчитает среднюю продолжительности сессии на сайте за 2023 год;

 SELECT AVG(session_time) AS avg_session_time
 FROM
 (SELECT play_session_id
         ,MAX(event_timestamp) - MIN(event_timestamp) as session_time
  FROM event_log
  WHERE event_timestamp BETWEEN '2023-01-01' AND '2024-01-01'
  GROUP BY play_session_id) AS play_session_info;

● который посчитает количество уникальных пользователей, посетивших сайт (DAU) по месяцам
Аналитика нужна за период с Июля-2023 по Декабрь-2023 (включительно);

 SELECT toMonth(event_timestamp) AS month
       ,COUNT(distinct player_id) AS coun_players
 WHERE player_id in
 (SELECT player_id
 FROM user_info
 WHERE email_verified != '1970-01-01')
 AND event_timestamp BETWEEN '2023-07-01' AND '2024-01-01'
 GROUP BY month, 
 ORDER BY month;

● который позволит оценить долю (по сумме успешного депозита) пользователей с подтвержденной почтой, в Декабре 2023 года;

 WITH confirmed_mail AS (
 SELECT distinct player_id
 FROM user_info
 WHERE email_verified != '1970-01-01'
 )

 SELECT 
 SUM(amount_RUB) FILTER (WHERE player_id IN (SELECT player_id FROM confirmed_mail))/
 SUM(amount_RUB) AS share_users
 FROM payment_records
 WHERE 
 payment_completed BETWEEN '2023-12-01' AND '2024-01-01'
 AND payment_successful=1;

● который выведет для каждого пользователя среднее значение, нижний квартиль, медиану и верхний квартиль по времени между
  успешными депозитами.

 WITH successful_deposits AS (
    SELECT 
        player_id
        ,payment_completed
    FROM payment_records
    WHERE payment_successful = 1
    ORDER BY player_id, payment_completed
 ),
 time_differences AS (
    SELECT 
       player_id
      ,payment_completed - lag(payment_completed, 1) OVER (PARTITION BY player_id ORDER BY payment_completed) AS successful_time
    FROM successful_deposits
    WHERE lag(payment_completed, 1) OVER (PARTITION BY player_id ORDER BY payment_completed) IS NOT NULL
 )
 
 
 SELECT 
    player_id
    ,AVG(successful_time) AS avg_time_between_deposits
    ,quantiles(0.25, 0.5, 0.75)(successful_time) AS quantiles
 FROM time_differences
 GROUP BY player_id;